In [1]:
import pandas as pd
import pickle

In [2]:
# Loading test data
test_df = pd.read_csv('../data_processed/model_data/test_data.csv')
val_df = pd.read_csv('../data_processed/model_data/val_data.csv')
test_df = pd.concat([test_df, val_df], ignore_index=True)
test_df['label'] = test_df['label'].apply(lambda x: [x])
X_test = test_df['text'].values.tolist()
y_test = test_df['label'].values.tolist()

Testing logistic regression with multi-label output

In [4]:
# Loading required models

vect_filepath = '../models/tf_idf_vec.sav'
tf_idf_vec = pickle.load(open(vect_filepath, 'rb'))

model_filepath = '../models/logistic_reg.sav'
log_reg_clf = pickle.load(open(model_filepath, 'rb'))

mlb_path = '../models/mlb.pkl'
mlb = pickle.load(open(mlb_path, 'rb'))

C:\Users\User\AppData\Local\Temp\ipykernel_7996\3097125959.py:4: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tf_idf_vec = pickle.load(open(vect_filepath, 'rb'))
C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnin

In [5]:
X_test = tf_idf_vec.transform(X_test).toarray()
y_test = mlb.transform(y_test)

In [6]:
y_test[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [7]:
y_pred = log_reg_clf.predict(X_test)

Для оценки работы мульти-лейб классификатора можно использовать следующие метрики:
- Exact Match Ratio - считает долю правильно предсказанных случаев, при условии, что частично-верно предсказанные примеры рассматриваются как неправильные
- 0/1 Loss - считает долю неправильно предсказанных случаев
- Accuracy - точность модели - средняя величина точности для всех экземпляров, доля правильных предсказаний
- Hamming Loss - считает сколько раз в среднем пример был неверно классифицирован. При подсчете используется как неправильное предсказание, так и пропуск предсказания, нормализованные по общему числу примеров и классов. (Чем ниже hamming loss, тем лучше работает модель)
- Recall - доля правильно предсказанных примеров к общему числу истинных значений, усредненная для всех значений
- Precision - доля правильно предсказанных примеров к общему числу предсказанных примеров, усредненная для всех значений
- F1-Measure - гармоническое среднее для Precision и Recall

In [8]:
import sklearn.metrics

print('Exact Match Ratio: {0}'.format(sklearn.metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)))

print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(y_test, y_pred))) 


Exact Match Ratio: 0.5030045434559578
Hamming loss: 0.02816209878352631


In [9]:
#"samples" applies only to multilabel problems. It does not calculate a per-class measure, instead calculating the metric over the true and predicted classes 
#for each sample in the evaluation data, and returning their (sample_weight-weighted) average.

print('Recall: {0}'.format(sklearn.metrics.precision_score(y_true=y_test, y_pred=y_pred, average='samples'))) 

print('Precision: {0}'.format(sklearn.metrics.recall_score(y_true=y_test, y_pred=y_pred, average='samples')))

print('F1 Measure: {0}'.format(sklearn.metrics.f1_score(y_true=y_test, y_pred=y_pred, average='samples'))) 

Recall: 0.5083540964385168
Precision: 0.5137036494210758
F1 Measure: 0.5101372807660365


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Я думаю, что нет смысла использовать average='samples', потому что в нашей тестовой выборке нет объектов, которые могут относиться сразу нескольким классам.

Поэтому лучше использовать 'macro' или 'micro' average

In [10]:
print('Recall macro: {0}'.format(sklearn.metrics.precision_score(y_true=y_test, y_pred=y_pred, average='macro'))) 

print('Precision macro: {0}'.format(sklearn.metrics.recall_score(y_true=y_test, y_pred=y_pred, average='macro')))
# невзвешенное среднее f1 для классов (считает f1 для каждого класса)
print('F1 Measure macro: {0}'.format(sklearn.metrics.f1_score(y_true=y_test, y_pred=y_pred, average='macro'))) 

Recall macro: 0.8546121274839921
Precision macro: 0.5137232435191066
F1 Measure macro: 0.6321161267552146


In [11]:
print('Recall micro: {0}'.format(sklearn.metrics.precision_score(y_true=y_test, y_pred=y_pred, average='samples'))) 

print('Precision micro: {0}'.format(sklearn.metrics.recall_score(y_true=y_test, y_pred=y_pred, average='samples')))
# считает общее число True Positives (TP), False Positives (FP) and False Negatives (FN) (не для каждого класса)
print('F1 Measure micro: {0}'.format(sklearn.metrics.f1_score(y_true=y_test, y_pred=y_pred, average='micro'))) 

Recall micro: 0.5083540964385168
Precision micro: 0.5137036494210758
F1 Measure micro: 0.6459043582419608


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Для несбалансированной выборки лучше использовать macro-f1, тк он дает каждому классу одинаковую важность. 
В нашем случае, выборка сбалансированная и мы можем использовать micro-f1, где каждый пример имеет равную важность. Также, micro-f1 можно интерпретировать как accuracy.

Также можно получить поклассово значения precision, recall и f1

In [12]:
f1_scores = sklearn.metrics.f1_score(y_test, y_pred, average=None)
precision_scores = sklearn.metrics.precision_score(y_test, y_pred, average=None)
recall_scores = sklearn.metrics.recall_score(y_test, y_pred, average=None)
df_accuracy = pd.DataFrame({"label": mlb.classes_,
                            # "auc": auc_scores,
                            'f1-score': f1_scores,
                            'precision': precision_scores,
                            'recall': recall_scores})
#df_accuracy = df_accuracy.sort_values('f1-score')[::-1]


In [13]:
df_accuracy

,label,f1-score,precision,recall
0,CASB,0.603571,0.728448,0.515244
1,EDR,0.509091,0.765273,0.381410
2,MDR,0.672384,0.843011,0.559201
3,NDR,0.818636,0.946154,0.721408
4,NGFW,0.742397,0.882979,0.640432
5,SASE,0.533088,0.736041,0.417867
6,SIEM,0.479843,0.769716,0.348571
7,SOAR,0.673816,0.884977,0.544012
8,anti-counterfeit,0.780834,0.958606,0.658683
9,application_control,0.797649,0.989583,0.668073


In [14]:
df_accuracy.to_csv('results/log_res_classes.csv', index=False)

Далее буду использовать только micro-average и результаты по классам

In [42]:
model_filepath = '../models/dec_tree.sav'
dec_tree_clf = pickle.load(open(model_filepath, 'rb'))

C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator MultiOutputClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [43]:
y_pred = dec_tree_clf.predict(X_test)

In [24]:
print('Exact Match Ratio: {0}'.format(sklearn.metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)))

print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(y_test, y_pred))) 

Exact Match Ratio: 0.7394840979041477
Hamming loss: 0.018800381064048074


In [25]:
print('Recall micro: {0}'.format(sklearn.metrics.precision_score(y_true=y_test, y_pred=y_pred, average='samples'))) 

print('Precision micro: {0}'.format(sklearn.metrics.recall_score(y_true=y_test, y_pred=y_pred, average='samples')))
# считает общее число True Positives (TP), False Positives (FP) and False Negatives (FN) (не для каждого класса)
print('F1 Measure micro: {0}'.format(sklearn.metrics.f1_score(y_true=y_test, y_pred=y_pred, average='micro'))) 

Recall micro: 0.763837998925204
Precision micro: 0.7893155503444232
F1 Measure micro: 0.8076331871180595


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
f1_scores = sklearn.metrics.f1_score(y_test, y_pred, average=None)
precision_scores = sklearn.metrics.precision_score(y_test, y_pred, average=None)
recall_scores = sklearn.metrics.recall_score(y_test, y_pred, average=None)
df_accuracy = pd.DataFrame({"label": mlb.classes_,
                            # "auc": auc_scores,
                            'f1-score': f1_scores,
                            'precision': precision_scores,
                            'recall': recall_scores})
#df_accuracy = df_accuracy.sort_values('f1-score')[::-1]


In [49]:
df_accuracy

,label,f1-score,precision,recall
0,CASB,0.734446,0.731118,0.737805
1,EDR,0.657829,0.680961,0.636218
2,MDR,0.802661,0.832822,0.774608
3,NDR,0.854950,0.896940,0.816716
4,NGFW,0.854415,0.881773,0.828704
5,SASE,0.748044,0.818493,0.688761
6,SIEM,0.739645,0.766871,0.714286
7,SOAR,0.763218,0.813725,0.718615
8,anti-counterfeit,0.950746,0.947917,0.953593
9,application_control,0.805694,0.815562,0.796062


In [50]:
df_accuracy.to_csv('results/dec_tree_classes.csv', index=False)

NN evaluation

Я сохранила только 5-ю эпоху, как наилучшую, поэтому буду получать метрики по ней. В файле test_nn выполнен код